In [1]:
import re
from glob import glob
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
top_dir = '/Users/emcmaho7/Dropbox/projects/SI_fmri/SIfMRI_modeling'
figures_dir = f'{top_dir}/reports/figures'
interim_dir = f'{top_dir}/data/interim'
raw_dir = f'{top_dir}/data/raw'

In [3]:
catch_trials = ['flickr-0-5-7-5-4-0-7-0-2605754070_54.mp4', 'yt-dfOVWymr76U_103.mp4']

In [4]:
caption_files = glob(f'{raw_dir}/sentence_captions/*.csv')

In [5]:
data = []
for path in caption_files:
    match = re.search(r'sub-(.*?)_condition-(.*?)_(.*).csv', path)
    if match:
        sub_id = match.group(1)
        condition = match.group(2)
        date = match.group(3)
        print(f"sub_id: {sub_id}, condition: {condition}, date: {date}")

    if os.path.getsize(path) == 0:
        print('oops that file is empty. moving on...')
    else:
        df = pd.read_csv(path, header=None)
        df.columns = ['url', 'caption']
        df['url'] = df['url'].str.extract("(https://[^']+)")
        df['video_name'] = df['url'].str.extract(r'/([^/]+\.mp4)')[0]
        df[['sub_id', 'condition', 'date']] = sub_id, condition, date
        data.append(df)
data = pd.concat(data)

sub_id: 606194ac4588bc4748d83964, condition: 01, date: 20230831142848
sub_id: 5efa66c030577f1782e0311a, condition: 08, date: 
oops that file is empty. moving on...
sub_id: 5cf15c363858150001955011, condition: 01, date: 20230831142825
sub_id: 5f7886db76d26207cd969fe6, condition: 22, date: 20230831140804
sub_id: 5c3f0188ba40520001d33fc2, condition: 01, date: 20230831142811
sub_id: 5f1e1924c9f24d7af7632e2e, condition: 01, date: 20230831142833
sub_id: 60b7ae30dc4cd72192734427, condition: 01, date: 20230831142837
sub_id: 5b5a4c642696a70001ee1722, condition: 00, date: 20230831142739
sub_id: 5a4d194c87c6f8000137d680, condition: 00, date: 20230831142734
sub_id: 5f4380bb7ca9ca5ca407fc04, condition: 04, date: 20230831143231
sub_id: 60bbe5dac754ff6671006e16, condition: 01, date: 20230831142842
sub_id: 5d5ad9a38fd0020001d2403f, condition: 01, date: 20230831142845
sub_id: 5d3112122401af0018ba2919, condition: 01, date: 20230831142821
sub_id: 59ad56559fdf6c0001cba91c, condition: 01, date: 20230831142

In [6]:
incomplete_data = data.groupby('sub_id').filter(lambda x: len(x) != 12)
filtered_data = data.groupby('sub_id').filter(lambda x: len(x) == 12)

In [7]:
for catch_trial in catch_trials:
    catch_data = filtered_data.loc[filtered_data.video_name == catch_trial].reset_index(drop=True)
    

In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('model_name_or_path')

ModuleNotFoundError: No module named 'sentence_transformers'